In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

### Functions

In [114]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
    except Error as e:
        print('Error '+e+' occured')
    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        #print("Query executed successfully")
    except Error as e:
        print('Error '+e+' occured')
        
def write_create_query(filename,col_name,col_type):
    create_table = "CREATE TABLE IF NOT EXISTS {0} (".format(filename)
    for i in range(len(col_name)):
        add_text = "{0} {1}".format(col_name[i],col_type[i])
        create_table = create_table + add_text
        if(i<len(col_name)-1):
            create_table = create_table + ','
        else:
            create_table = create_table + ')'
    return create_table

def select_query(conn,query):
    
    cur = conn.cursor()
    cur.execute(query)
    
    rows = cur.fetchall()
    
    return rows

def insert_query(connection,list_value,filename,col_type):
    insert_table = "INSERT INTO {0}(".format(filename)
    for i in range(len(col_name)):
        add_text = "{0}".format(col_name[i])
        insert_table = insert_table + add_text
        if(i<len(col_name)-1):
            insert_table = insert_table + ','
        else:
            insert_table = insert_table + ')'

    query_val = ' VALUES ('
    for idx,val in enumerate(list_value):
        if(col_type[idx]=='TEXT'):
            val = "'"+val+"'"

        query_val +=  val
        if(idx<len(list_value)-1):
            query_val += ','
        else:
            query_val += ')'

    return insert_table + query_val

def choose_type(dtype):
    if dtype=='object':
        return 'TEXT'
    if 'int' in dtype:
        return 'INT'
    elif 'float' in dtype:
        return 'FLOAT'

## Create Connection

In [8]:
connection = create_connection('db.sqlite')

## Load data with Pandas

In [156]:
filename = 'studentInfo'
df=pd.read_csv(filename+'.csv')
df=df[df.isna().sum(axis=1)==0]

In [157]:
df.head(10)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass
5,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass
6,AAA,2013J,45462,M,Scotland,HE Qualification,30-40%,0-35,0,60,N,Pass
7,AAA,2013J,45642,F,North Western Region,A Level or Equivalent,90-100%,0-35,0,120,N,Pass
8,AAA,2013J,52130,F,East Anglian Region,A Level or Equivalent,70-80%,0-35,0,90,N,Pass
10,AAA,2013J,57506,M,South Region,Lower Than A Level,70-80%,35-55,0,60,N,Pass


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31482 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           31482 non-null  object
 1   code_presentation     31482 non-null  object
 2   id_student            31482 non-null  int64 
 3   gender                31482 non-null  object
 4   region                31482 non-null  object
 5   highest_education     31482 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              31482 non-null  object
 8   num_of_prev_attempts  31482 non-null  int64 
 9   studied_credits       31482 non-null  int64 
 10  disability            31482 non-null  object
 11  final_result          31482 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.1+ MB


### Create Table

In [117]:
execute_query(connection, "drop table if exists "+filename)

Query executed successfully


Read columns and designate column type

In [162]:
col_name = df.columns.values.tolist()
col_type = [choose_type(i.name) for i in df.dtypes.values]

In [163]:
create_query = write_create_query(filename,col_name,col_type)

In [164]:
create_query

'CREATE TABLE IF NOT EXISTS studentInfo (code_module TEXT,code_presentation TEXT,id_student INT,gender TEXT,region TEXT,highest_education TEXT,imd_band TEXT,age_band TEXT,num_of_prev_attempts INT,studied_credits INT,disability TEXT,final_result TEXT)'

In [121]:
execute_query(connection, create_query)

Query executed successfully


### Insert to Table

In [122]:
for k in range(len(df)):
    list_value = df.iloc[k].values.astype(str).tolist()
    query = insert_query(connection,list_value,filename,col_type)
    execute_query(connection,query)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


# Wrap Creating table into single function

In [130]:
file_list = ['assessments','courses','studentAssessment','studentInfo','vle']

In [136]:
for filename in file_list[3:]:
    
    # Read csv file
    df=pd.read_csv(filename+'.csv')
    df=df[df.isna().sum(axis=1)==0]

    # drop table if exists to prevent overwrite error
    execute_query(connection, "drop table if exists "+filename)

    # get original column name and appropriate data type
    col_name = df.columns.values.tolist()
    col_type = [choose_type(i.name) for i in df.dtypes.values]

    # Create and execute query to create new table in sqlite
    create_query = write_create_query(filename,col_name,col_type)
    execute_query(connection, create_query)

    # Insert data to corresponding table in sqlite
    for k in range(len(df)):
        list_value = df.iloc[k].values.astype(str).tolist()
        query = insert_query(connection,list_value,filename,col_type)
        execute_query(connection,query)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

...

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

In [ ]:
for filename in file_list:
    # Retrive table from sqlite
    Q = "SELECT * from {0} where code_presentation = '2014B'".format(filename)
    rows = select_query(connection,Q)

    # create DataFrame using retrieved data
    df_filtered = pd.DataFrame(rows, columns =df.columns)

    # Write dataframe to excel
    df_filtered.to_excel(filename+'.xlsx',index=False)

## Create Select Query and Retrieve Data

In [142]:
cursor = connection.execute('select * from assessments')
print(list(map(lambda x: x[0], cursor.description)))

['code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'weight']


In [143]:
cursor = connection.execute('select * from courses')
print(list(map(lambda x: x[0], cursor.description)))

['code_module', 'code_presentation', 'module_presentation_length']


In [144]:
cursor = connection.execute('select * from studentAssessment')
print(list(map(lambda x: x[0], cursor.description)))

['id_assessment', 'id_student', 'date_submitted', 'is_banked', 'score']


In [145]:
cursor = connection.execute('select * from studentInfo')
print(list(map(lambda x: x[0], cursor.description)))

['code_module', 'code_presentation', 'id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result']


In [146]:
cursor = connection.execute('select * from vle')
print(list(map(lambda x: x[0], cursor.description)))

['id_site', 'code_module', 'code_presentation', 'activity_type', 'week_from', 'week_to']


In [180]:
Q = '''select aa.id_student,region,cc.code_module,cc.assessment_type,bb.id_assessment,bb.score from studentInfo as aa
                            left join studentAssessment as bb on aa.id_student = bb.id_student
                            left join assessments as cc on bb.id_assessment = cc.id_assessment
                            where aa.code_presentation='2014B' and cc.code_presentation='2014B'
                            '''
rows = select_query(connection,Q)

In [181]:
rows

[(25997, 'London Region', 'BBB', 'TMA', 15009, 73.0),
 (25997, 'London Region', 'BBB', 'CMA', 15015, 100.0),
 (27891, 'Scotland', 'BBB', 'TMA', 15008, 73.0),
 (27891, 'Scotland', 'BBB', 'TMA', 15009, 67.0),
 (27891, 'Scotland', 'BBB', 'TMA', 15010, 55.0),
 (27891, 'Scotland', 'BBB', 'CMA', 15015, 100.0),
 (27891, 'Scotland', 'BBB', 'CMA', 15016, 60.0),
 (27891, 'Scotland', 'BBB', 'CMA', 15017, 60.0),
 (38941, 'East Midlands Region', 'BBB', 'TMA', 15008, 77.0),
 (50069, 'Scotland', 'BBB', 'TMA', 15008, 78.0),
 (50069, 'Scotland', 'BBB', 'TMA', 15009, 84.0),
 (50069, 'Scotland', 'BBB', 'TMA', 15011, 67.0),
 (50069, 'Scotland', 'BBB', 'CMA', 15015, 100.0),
 (50069, 'Scotland', 'BBB', 'CMA', 15016, 100.0),
 (50069, 'Scotland', 'BBB', 'CMA', 15017, 80.0),
 (52426, 'Scotland', 'BBB', 'TMA', 15008, 46.0),
 (52426, 'Scotland', 'BBB', 'TMA', 15009, 66.0),
 (52426, 'Scotland', 'BBB', 'TMA', 15010, 53.0),
 (52426, 'Scotland', 'BBB', 'CMA', 15015, 100.0),
 (52426, 'Scotland', 'BBB', 'CMA', 15016, 

### Convert Result to Dataframe and write to excel

In [182]:
# create DataFrame using data
df_filtered = pd.DataFrame(rows, columns =['id_student','region','code_module','assessment_type','id_assessment','score'])

In [183]:
df_filtered.to_excel('results.xlsx',index=False)